In [1]:
#prompt = "Given the following list of adverse events, write 1 if present in note, 0 if it is not. Return a list of pairs"

In [2]:
prompt = "Determine if the adverse event is present in the note. Let’s think step by step. The result should be a JSON object. The JSON object has two properties [“class”, “explanation”]. The “class” is an integer field, 0 indicates adverse event is not present, and 1 indicates adverse event is present. The “explanation” should be the reasoning steps. The property name in the JSON object should be enclosed in double quotes."

In [3]:
ade_dic = {
    "Neutropenia": 0,
    "Anemia": 0,
    "Thrombocytopenia": 0,
    "Febrile Neutropenia": 0,
    "Upper Respiratory Tract Infection": 0,
    "Urinary Tract Infection": 0,
    "Sinusitis": 0,
    "Cellulitis": 0,
    "Pneumonia": 0,
    "Fatigue": 0,
    "Rash": 0,
    "Fever": 0,
    "Cough": 0,
    "Diarrhea": 0,
    "Hyperglycemia": 0,
    "Constipation": 0,
    "Edema": 0,
    "Inflammatory Syndrome": 0,
    "Infusion Reaction": 0,
    "Nausea": 0,
    "Anorexia": 0,
    "Dyspnea": 0,
    "Elevated alanine aminotransferase level": 0,
    "Elevated aspartate aminotransferase level": 0,
    "Hypothyroidism": 0
}


# Convert the list of tuples to a string
my_list = list(ade_dic.items())
result_string = ', '.join(f"{key}: {value}" for key, value in my_list)
print(result_string)
header = list(ade_dic.keys())
header_as_string = ', '.join(f"{value}" for value in header)
print(header_as_string)

Neutropenia: 0, Anemia: 0, Thrombocytopenia: 0, Febrile Neutropenia: 0, Upper Respiratory Tract Infection: 0, Urinary Tract Infection: 0, Sinusitis: 0, Cellulitis: 0, Pneumonia: 0, Fatigue: 0, Rash: 0, Fever: 0, Cough: 0, Diarrhea: 0, Hyperglycemia: 0, Constipation: 0, Edema: 0, Inflammatory Syndrome: 0, Infusion Reaction: 0, Nausea: 0, Anorexia: 0, Dyspnea: 0, Elevated alanine aminotransferase level: 0, Elevated aspartate aminotransferase level: 0, Hypothyroidism: 0
Neutropenia, Anemia, Thrombocytopenia, Febrile Neutropenia, Upper Respiratory Tract Infection, Urinary Tract Infection, Sinusitis, Cellulitis, Pneumonia, Fatigue, Rash, Fever, Cough, Diarrhea, Hyperglycemia, Constipation, Edema, Inflammatory Syndrome, Infusion Reaction, Nausea, Anorexia, Dyspnea, Elevated alanine aminotransferase level, Elevated aspartate aminotransferase level, Hypothyroidism


In [4]:
PROJECT_ID = "vertex-aashri4026-04e5"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [5]:
REGION = "us-central1"  # @param {type: "string"}

In [6]:
BUCKET_NAME = f"your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [7]:
! gsutil mb -l {REGION} {BUCKET_URI}

Creating gs://your-bucket-name-vertex-aashri4026-04e5-unique/...
ServiceException: 409 A Cloud Storage bucket named 'your-bucket-name-vertex-aashri4026-04e5-unique' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [8]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models
 
def generate(ade, note):
    total_text = ""
    vertexai.init(project="vertex-aashri4026-04e5", location="us-central1")
    model = GenerativeModel("gemini-1.0-pro-001")
    try:
        responses = model.generate_content(
            [prompt, ade, note],
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream=True,
        )
    
        for response in responses:
            if not response.candidates:
                print("No candidates found for response")
                continue
            try:
                #print(response.text, end="")
                total_text = total_text + response.text
            except AttributeError:
                print("In Generate Function, Error: Content has no parts.")
                #print(ade)
    except: total_text = 'Model Failed'      
    return total_text
 
generation_config = {
    "max_output_tokens": 8000,
    "temperature": 1,
    "top_p": 0.95,
}
# generation_config = {
#     "max_output_tokens": 8000,
#     "temperature": 0,
#     "top_p": 1,
# }
 
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}


In [ ]:
import os

folder_path = r"RedCapNotes_905"
annotations = []
response_dic = {}
i = 1
header = list(ade_dic.keys())
for filename in os.listdir(folder_path):
    note_dic = {}
    print(i)
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            note = file.read()
            note_name = filename[:-4]
            # print(note_name)
            j=1
            for ade in header:
                #print("This is j")
                #print(ade)
                #print(j)
                try:
                    note_annotation = generate("Adverse Event: " + ade,note)
                except AttributeError:
                    #print(ade)
                    #print(note)
                    note_annotation = ade_dic
                note_dic[ade] = note_annotation
                j+=1          
    print(note_dic)
    response_dic[note_name] = note_dic
    i+=1
            

In [ ]:
import json

#data = response_dic["Z1417573"]
def parse_values(note):
    data = response_dic[note]
    parsed_values = {}
    for key, value in data.items():
        if value.startswith('```json') and value.endswith('```'):
            cleaned_value = value.replace('```json','').replace('```','')
        elif value.startswith('```') and value.endswith('```'):
            cleaned_value = value.replace('```','').replace('```','')
        else:
            cleaned_value = value.replace("\n","").replace(" ","")
        try:
            parsed_json = json.loads(cleaned_value)
            parsed_values[key]=parsed_json['class']
        except Exception as e:
            print(key)
            print(value)
    return parsed_values

In [ ]:
import csv

# Open the CSV file in write mode
myFile = open('redcap_real_906_medpalm.csv', 'w')

# Create a csv.writer object
writer = csv.writer(myFile)

# Write the header row (keys of the dictionary)
header = list(ade_dic.keys())
header.insert(0, "note_name")
writer.writerow(header)

# Write each dictionary as a row
for note in response_dic:
    note_name = note
    parsed_values = parse_values(note)
    values = list(parsed_values.values())
    output = [str(x) for x in values]
    output.insert(0, note_name)
    writer.writerow(output)

# Close the file
myFile.close()


In [1]:
#Save all the annotations and output
def parse_json(note):
    data = response_dic[note]
    parsed_values = {}
    for key, value in data.items():
        if value.startswith('```json') and value.endswith('```'):
            cleaned_value = value.replace('```json','').replace('```','')
        elif value.startswith('```') and value.endswith('```'):
            cleaned_value = value.replace('```','').replace('```','')
        else:
            cleaned_value = value.replace("\n","")
        try:
            parsed_json = json.loads(cleaned_value)
            parsed_values[key]=parsed_json
        except Exception as e:
            print(key)
            print(value)
    return parsed_values

folder_path = "Annotated_RedCap_905_medpalm"
os.makedirs(folder_path, exist_ok=True)

for note in response_dic:
    note_name = note
    output_file_path = os.path.join(folder_path, note_name) 
    data = parse_json(note)
    json_data = json.dumps(data, ensure_ascii=False, indent=4)
    with open(output_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(json_data)

NameError: name 'os' is not defined

In [2]:
import zipfile
import os

def create_zip(folder_path, output_zip_filename):
    """
    Creates a ZIP file containing all files and subfolders in the specified folder.

    Args:
        folder_path (str): Path to the folder you want to zip.
        output_zip_filename (str): Name of the output ZIP file.

    Returns:
        None
    """
    with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            i = 0
            for file in files:
                print(i)
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=arcname)
                i+=1

# Example usage
folder_to_zip = r'Annotated_RedCap_905_medpalm'
output_zip_file = 'Annotated_RedCap_905_medpalm.zip'
create_zip(folder_to_zip, output_zip_file)
print(f"ZIP file '{output_zip_file}' created successfully.")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27